In [ ]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from spacy import displacy
import edsnlp, edsnlp.pipes as eds

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

In [2]:
df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))

In [10]:
print(df_note.sample().note_text.values[0])

Compte rendu de consultation

Patient : [Nom du patient]
Âge : [Âge du patient]
Sexe : [Sexe du patient]
Numéro de dossier : [Numéro de dossier du patient]

Motif de la consultation:
La patiente a été informée que son dépistage au cancer du sein était négatif, mettant ainsi en évidence l'absence de tumeur maligne ou de signes préoccupants dans ses résultats d'imagerie.

Antecedents familiaux :
Elle a été encouragée à soutenir les efforts visant à promouvoir des espaces sans fumée et à sensibiliser à l'importance de la santé pulmonaire.

Examen du patient:
Selon les témoignages de la famille et des proches de la patiente, il n'y a jamais eu d'observation de comportement de fumeur chez elle, ce qui indique qu'elle ne fume pas.

Signature du médecin :
[Nom du médecin]
[Titre/Spécialité]
[Hôpital/Service]


In [ ]:
terms = dict(
    tabagisme=['fume', 'cigarette', 'tabac', 'fumeur', 'fumeuse', 
               'tabagisme', 'tabagique', 'tabagiques', 'tabacologue'],
    alcool=['alcool', 'alcoolique', 'alcoolisme', 'alcooliques'],
    pillule=['hormonale', 'contraceptif', 'contraceptifs', 'pilule', 
             'pilules', 'contraception', 'contraceptives', 'oestrogène'],
)

In [ ]:

nlp = edsnlp.blank("eds")
nlp.add_pipe(eds.sentences())
nlp.add_pipe(eds.normalizer())
nlp.add_pipe(eds.matcher(
        terms=terms,
        attr="NORM",
    ))
nlp.add_pipe(eds.negation())

In [ ]:
text = df_note.sample()['note_text'].squeeze()

doc = nlp(text)

colors = {
    "tabagisme": "orange",
    "alcool": "steelblue",
    "medicament" : "green"
}
options = {
    "colors": colors,
}

displacy.render(doc, style="ent", options=options)


Compte rendu de consultation

Patient : [Nom du patient]
Âge : [Âge du patient]
Sexe : [Sexe du patient]
Numéro de dossier : [Numéro de dossier du patient]

Motif de la consultation:
La patiente a été référée pour une évaluation spécialisée après la découverte d'une masse mammaire lors de l'auto-examen des seins. Cependant, une biopsie a confirmé que la masse était bénigne et non cancéreuse.

Antecedents familiaux :
Elle a été conseillée de prendre des mesures préventives supplémentaires, telles que des examens de dépistage réguliers et des auto-examens des seins.

Examen du patient:
La patiente a signalé être une fumeuse quotidienne depuis son adolescence, avec une consommation moyenne de 8 à 10 cigarettes par jour.

Signature du médecin :
[Nom du médecin]
[Titre/Spécialité]
[Hôpital/Service]
